# Batch consuming

In [ ]:
# | hide

import asyncio

import asyncer
from IPython.display import Markdown as md

from fastkafka._components._subprocess import terminate_asyncio_process
from fastkafka._testing.apache_kafka_broker import run_and_match
from fastkafka.testing import ApacheKafkaBroker, run_script_and_cancel

If you want to consume data in batches `@consumes` decorator makes that possible for you. By typing a consumed msg object as a `list` of messages the consumer will call your consuming function with a batch of messages consumed from a single partition. Let's demonstrate that now.

## Consume function with batching

To consume messages in batches, you need to wrap you message type into a list and the `@consumes` decorator will take care of the rest for you. Your consumes function will be called with batches grouped by partition now.

In [ ]:
# | echo: false

consumes_decorator_batch = """@app.consumes(auto_offset_reset="earliest")
async def on_hello_world(msg: List[HelloWorld]):
    logger.info(f"Got msg batch: {msg}")
"""
md(f"```python\n{consumes_decorator_batch}\n```")

```python
@app.consumes(auto_offset_reset="earliest")
async def on_hello_world(msg: List[HelloWorld]):
    logger.info(f"Got msg batch: {msg}")

```

## App example

We will modify the app example from [@consumes basics](/docs/guides/Guide_11_Consumes_Basics.md) guide to consume `HelloWorld` messages batch. The final app will look like this (make sure you replace the `<url_of_your_kafka_bootstrap_server>` and `<port_of_your_kafka_bootstrap_server>` with the actual values):

In [ ]:
# | hide

app = """
import asyncio
from typing import List
from pydantic import BaseModel, Field

from fastkafka import FastKafka
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)

"""

In [ ]:
# | echo: false

batch_example = app + consumes_decorator_batch

md(f"```python\n{batch_example}\n```")

```python

import asyncio
from typing import List
from pydantic import BaseModel, Field

from fastkafka import FastKafka
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

class HelloWorld(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_brokers = {
    "demo_broker": {
        "url": "<url_of_your_kafka_bootstrap_server>",
        "description": "local demo kafka broker",
        "port": "<port_of_your_kafka_bootstrap_server>",
    }
}

app = FastKafka(kafka_brokers=kafka_brokers)

@app.consumes(auto_offset_reset="earliest")
async def on_hello_world(msg: List[HelloWorld]):
    logger.info(f"Got msg batch: {msg}")

```

## Send the messages to kafka topic

Lets send a couple of `HelloWorld` messages to the *hello_world* topic and check if our consumer kafka application has logged the received messages batch. In your terminal, run the following command at least two times to create multiple messages in your kafka queue:

In [ ]:
# | echo: false

producer_cmd = 'echo {\\"msg\\": \\"Hello world\\"} | kafka-console-producer.sh --topic=hello_world --bootstrap-server=<addr_of_your_kafka_bootstrap_server>'
md(f"```shell\n{producer_cmd}\n```")

```shell
echo {\"msg\": \"Hello world\"} | kafka-console-producer.sh --topic=hello_world --bootstrap-server=<addr_of_your_kafka_bootstrap_server>
```

In [ ]:
# | echo: false

script_file = "consumer_example.py"
filename = script_file.split(".py")[0]
cmd = f"fastkafka run --num-workers=1 --kafka-broker=demo_broker {filename}:app"
md(
    f"Now we can run the app. Copy the code of the example app in {script_file} and run it by running\n```shell\n{cmd}\n```"
)

Now we can run the app. Copy the code of the example app in consumer_example.py and run it by running
```shell
fastkafka run --num-workers=1 --kafka-broker=demo_broker consumer_example:app
```

In [ ]:
# | hide


with ApacheKafkaBroker(
    topics=["hello_world"], apply_nest_asyncio=True
) as bootstrap_server:
    async with asyncer.create_task_group() as task_group:
        server_url = bootstrap_server.split(":")[0]
        server_port = bootstrap_server.split(":")[1]

        producer_tasks = [task_group.soonify(asyncio.create_subprocess_shell)(
            cmd=producer_cmd.replace(
                "<addr_of_your_kafka_bootstrap_server>", bootstrap_server
            ),
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        ) for _ in range(2)]
        
        await asyncio.sleep(5)
        
        consumer_task = task_group.soonify(run_script_and_cancel)(
            script=batch_example.replace(
                "<url_of_your_kafka_bootstrap_server>", server_url
            ).replace("<port_of_your_kafka_bootstrap_server>", server_port),
            script_file=script_file,
            cmd=cmd,
            cancel_after=20,
        )

assert "Got msg batch: [HelloWorld(msg='Hello world'), HelloWorld(msg='Hello world')]" in consumer_task.value[1].decode("UTF-8")

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.sta

In [ ]:
# | echo: False

print(consumer_task.value[1].decode("UTF-8"))

You should see the your Kafka messages being logged in batches by your consumer.